In [2]:
from sklearn import cluster
import networkx as nx
from collections import defaultdict
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics.cluster import adjusted_rand_score
import pickle

In [3]:
with open ('polemic_comments.pkl', 'rb') as fp:
    polemic_comments = pickle.load(fp)

In [12]:
"достать все комментарии в каждом кластере"
only_comments_by_cluster = {}
for clstr in polemic_comments:
    clstr_comments = []
    cntr_articles = 0
    for article in clstr[1]:
        cntr_articles += 1
        comments_arr = article[2]
        #print(len(comments_arr))
        if len(comments_arr) != 0:
            #print(comments_arr)
            for inner_tuple in comments_arr:
                #print(inner_tuple)
                for comment_text in inner_tuple[1]:
                    clstr_comments.append(comment_text)
    cntr_comments = len(clstr_comments)
    print(clstr[0], cntr_articles, cntr_comments, cntr_comments/cntr_articles)
    only_comments_by_cluster[clstr[0]] = clstr_comments

6 18 74 4.111111111111111
7 28 87 3.107142857142857
12 13 30 2.3076923076923075
16 7 31 4.428571428571429
20 32 443 13.84375
21 20 248 12.4
24 8 47 5.875
30 68 1521 22.36764705882353
33 17 985 57.94117647058823
34 16 334 20.875


In [9]:
import nltk, math, codecs
from gensim.models import Doc2Vec
from nltk.cluster.kmeans import KMeansClusterer
import re
import pymorphy2
from datetime import datetime
from sklearn.metrics.pairwise import cosine_distances

fname = 'noStopLemma_PV-DBOW_wrd-vec_1it_2win_6mincount_alpha25-25_sz80.model'

model = Doc2Vec.load(fname)

morph = pymorphy2.MorphAnalyzer()

C:\Anaconda\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [10]:
with open('stopwords.txt', 'r', encoding='utf-8') as f:
    txt = f.read().split('\n')
stw = set(txt)

In [13]:
def preprocess(str):
    # remove links
    str = re.sub(r'http(s)?:\/\/\S*? ', "", str)
    return str


def preprocess_document(text):
    #text = preprocess(text)
    fixedNoStop = []
    fixed = ''.join([x if x.isalnum() or x.isspace() else " " for x in text ]).split()
    for fix in fixed:
        if fix not in stw:
            fix = morph.parse(fix)[0].normal_form
            fixedNoStop.append(fix)
    return fixedNoStop

start_time = datetime.now()

#NUM_CLUSTERS = 15

#data = <sparse matrix that you would normally give to scikit>.toarray()

#corpus = codecs.open('test_headlines_short.txt', mode="r", encoding="utf-8")
#for cluster in only_comments_by_cluster:
#    print(len(only_comments_by_cluster[cluster]))
#lines = ['Бог даст - и во втором победит.', 'В не всякого сомнения, победа на выборах за Марин Ле Пен, какие бы козни её не чинили...', 'Как с Трампом. 61% из специальной выборки, исключающей 100% тех, кто точно против.', 'Макрон - Президент Франции! Трамп - ненаш! Черногория в НАТО! Обидно, да, рос-патриоты?))))', 'ещё не вечер . А этим заказным опросам грош цена в базарный день .', 'Ага, верю-верю! Клинтоншу тоже, согласно опросов, поддерживало большинство, и победа была гарантирована. И где теперь те опросники, и их опросы- прогнозы?))', 'После выборов,если будет ставленник пиндосии Макарон, то французов мне жалко. И больше они для меня не будут представлять Великую Францию.', 'Сергей Викторов, Ну, зачем ты так жестоко с ними? Они же очень сильно расстроятся, что не представляют для рос-патриота сереги Великую Францию)))))', 'До уличного светила, что называется, это дело народа Франции.', 'Опрос про Клинтон вспомнился...', 'Александр Михайлов,  Про Париж точно знаешь? Часто там бываешь, рос-патриот?))))', 'Александр Михайлов, Сколько боярышника принял, рос-патриот,  чтобы в соседнем дворе "Париж" увидеть?))))']
lines = only_comments_by_cluster['20']
for line in range(len(lines)):
    lines[line] = lines[line].lower()
#lines = corpus.read().lower().split('\r\n')
#count = len(lines)

vectors = []

print("inferring vectors")
duplicate_dict = {}
used_lines = []
for i, t in enumerate(lines):
    if t not in duplicate_dict:#i % 2 == 0 and
        duplicate_dict[t] = True
        used_lines.append(t)
        vectors.append(model.infer_vector(preprocess_document(t)))

print("done")

inferring vectors
done


In [44]:
distances = cosine_distances(vectors)

In [46]:
#algorithms['affinity'] = cluster.AffinityPropagation(damping=0.6)
affinity = cluster.AffinityPropagation(damping=0.6, affinity = 'precomputed')
affinity.fit(distances)
# Fit all models
#for model in algorithms.values():
#    model.fit(edge_mat)
#    results.append(list(model.labels_))

AffinityPropagation(affinity='precomputed', convergence_iter=15, copy=True,
          damping=0.6, max_iter=200, preference=None, verbose=False)

In [47]:
#result = []
result = list(affinity.labels_)
cluster_centers_indices = affinity.cluster_centers_indices_
labels = affinity.labels_
n_clusters_ = len(cluster_centers_indices)

In [59]:
cluster_centers_indices[0]

5

In [68]:
clustersizes = []

def distanceToCentroid():
    all_distances = []
    for i in range(0, NUM_CLUSTERS):
        clustersize = 0
        for j in range(0, len(result)):
            if (result[j] == i):
                clustersize+=1
        clustersizes.append(clustersize)
        dist = 0.0
        centr = kclusterer.means()[i]
        for j in range(0, len(assigned_clusters)):
            if (assigned_clusters[j] == i):
                dist += pow(nltk.cluster.util.cosine_distance(vectors[j], centr),2)/clustersize
        dist = math.sqrt(dist)
        all_distances.append(dist)
        #print("distance cluster: "+str(i)+" RMSE: "+str(dist)+" clustersize: "+str(clustersize))
    return all_distances

def nClosestToCentroid(cluster_id, n):
    #clustersize = len(get_titles_by_cluster(cluster_id))
    #centr = kclusterer.means()[cluster_id]
    centr = vectors[affinity.cluster_centers_indices_[cluster_id]]
    distances = []
    for j in range(0, len(result)):
        if (result[j] == cluster_id):
            distances.append((used_lines[j], nltk.cluster.util.cosine_distance(vectors[j], centr)))
            
    #print(distances)
    distances = sorted(distances, key=lambda tup: tup[1])
    return distances[:n]


def get_titles_by_cluster(id):
    list = []
    for x in range(0, len(result)):
        if (result[x] == id):
            list.append(used_lines[x])
    return list

def get_topics(titles):
    from collections import Counter
    words = [preprocess_document(x) for x in titles]
    words = [word for sublist in words for word in sublist]
    #filtered_words = [word for word in words if word not in stw]
    count = Counter(words)
    print(count.most_common()[:5])


def cluster_to_topics(id):
    get_topics(get_titles_by_cluster(id))


In [70]:
#cluster_to_topics(clstr)
for clstr in range(n_clusters_):
    #cluster_to_topics(clstr)
    #print(get_titles_by_cluster(clstr))
    n = round(len(get_titles_by_cluster(clstr))/3)
    print(n)
    print(nClosestToCentroid(clstr, n))
#print('####\n')

#посмотреть выдачу

30
[('мальчиш-плохиш совершенствует свою рогатку :-))', -2.2204460492503131e-16), ('замените пожалуйста в ех татжтков и узбеков на рабочих из кндр.', 1.0239170902249537), ('нужно расширить экспорт в кндр всей продукции. это деньги в нашу страну!', 1.0705560264308986), ('игорь ,, та ни шо.', 1.0706994404899213), ('kymapc, никто там ничем "жахать не будет. вы что серьезно не понимаете что там происходит? штатам выгодна напряженная обстановка рядом с китаем. чтобы иметь предлог для нахождения там своих военно-морских сил. и именно по этому они провоцируют северную корею на новые выпады, проводя там "учения", якобы для защиты союзников по нато.=вы вообще в курсе что ирак разрушили демократичные и цивилизованные страны под сфальсифицированным предлогом того что саддам может использовать оружие массового поражения. после чего на его территории и образовался игил. в результате тысячи невинных смертей, сотни тысяч беженцев.вы хотите чтобы мы с такими вот неадекватами дружили? которые уничтожаю

In [66]:
affinity.cluster_centers_indices_[0]

5